In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, random_split
import time

from horse_dataset import HorseDataset
from winning_set import WinningSetModule, send_next_layer

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
def train(dataset, model, optimizer, n_iters): 
    model.to(device=device)
    model.train()
    start = time.time()
    print_every = 10
    for e in range(n_iters):
        for i, (x, y) in enumerate(dataset):
            x = x.to(device=device)
            y = (len(y[0]) - y.type(torch.float32)) / (len(y[0]) - 1)
            y = (torch.exp(y) - 1) / 1.72
            y = y.to(device=device)
            model.zero_grad()
            output = model(x)
            loss = loss_fcn(output, y)
            loss.backward()
            optimizer.step()
        if e % print_every == 0:
            print('(%d %d%%) %.4f' % (e, e / n_iters * 100, loss))

In [3]:
def predict_helper(model, x):
    batch_size = len(x)
    output = model(x)
    result = [None for _ in range(batch_size)]
    recommand_lanes = torch.sort(torch.topk(output, 7, sorted=False)[1])[0]
    for index in range(batch_size):
        result[index] = x[index, recommand_lanes[index]]
    return torch.stack(tuple(result)), recommand_lanes


In [17]:
print('using device:', device)
loss_fcn = nn.MSELoss()

model = WinningSetModule(9, 8, 14)
_size = 8

n_iters = 30

test_data_saved = []
train_data_saved = []

for _size in range(8, 15):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    dataset = HorseDataset(f'./preprocess/data/data_{_size:02}.pkl')
    _dataset_size = len(dataset)
    _test_data_size = _dataset_size // 10
    _train_data_size = _dataset_size - _test_data_size
    
    #print([_train_data_size, _test_data_size])
    
    train_dataset, test_dataset = random_split(dataset, [_train_data_size, _test_data_size])
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    train(train_loader, model, optimizer, n_iters)
    
    test_data_saved.append(test_loader) # test를 추출합니다
    train_data_saved.append(train_loader) # train를 추출합니다

    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 7
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([sum([1 if (val in result[index]) else 0 for val in predicted[index]]) for index in range(len(x))]) / sc
        total += len(y)
    print(f'lane size: {_size}, top 7 accuarcy_1: {s / total}%, {total}')

    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 7
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([1 if (sum([1 if (val in result[index]) else 0 for val in predicted[index]]) == sc) else 0 for index in range(len(x))])
        total += len(y)
    print(f'lane size: {_size}, top 7 accuarcy_2: {s / total}%, {total}')
    
    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([sum([1 if (val in result[index]) else 0 for val in predicted[index]]) for index in range(len(x))]) / sc
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_1: {s / total}%, {total}')
    
    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([1 if (sum([1 if (val in result[index]) else 0 for val in predicted[index]]) == sc) else 0 for index in range(len(x))])
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_2: {s / total}%, {total}')


using device: cpu
[2094, 232]
(0 0%) 0.0959
(10 33%) 0.0668
(20 66%) 0.0654
lane size: 8, top 7 accuarcy_1: 0.9033251231527094%, 232
lane size: 8, top 7 accuarcy_2: 0.3232758620689655%, 232
lane size: 8, top 3 accuarcy_1: 0.9784482758620688%, 232
lane size: 8, top 3 accuarcy_2: 0.9353448275862069%, 232
[2703, 300]
(0 0%) 0.0941
(10 33%) 0.0806
(20 66%) 0.0620
lane size: 9, top 7 accuarcy_1: 0.8519047619047618%, 300
lane size: 9, top 7 accuarcy_2: 0.15%, 300
lane size: 9, top 3 accuarcy_1: 0.96%, 300
lane size: 9, top 3 accuarcy_2: 0.8833333333333333%, 300


In [10]:
# torch.save(model.state_dict(), 'fng_pt2.pt')

In [6]:
model = WinningSetModule(9, 8, 14)
model.load_state_dict(torch.load('fng_pt2.pt', map_location=device))
model = model.to(device=device)

In [7]:
for _size in range(8, 15):
    dataset = HorseDataset(f'./preprocess/data/data_{_size:02}.pkl')
    _loader = DataLoader(dataset, batch_size=32, shuffle=False)
    s = 0
    total = 0
    for i, (x, y) in enumerate(_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([sum([1 if (val in result[index]) else 0 for val in predicted[index]]) for index in range(len(x))]) / sc
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_1: {s / total}%, {total}')
    
    s = 0
    total = 0
    for i, (x, y) in enumerate(_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([1 if (sum([1 if (val in result[index]) else 0 for val in predicted[index]]) == sc) else 0 for index in range(len(x))])
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_2: {s / total}%, {total}')

lane size: 8, top 3 accuarcy_1: 0.9819432502149611%, 2326
lane size: 8, top 3 accuarcy_2: 0.945829750644884%, 2326
lane size: 9, top 3 accuarcy_1: 0.9615939615939615%, 3003
lane size: 9, top 3 accuarcy_2: 0.8867798867798867%, 3003
lane size: 10, top 3 accuarcy_1: 0.9440077632217366%, 3435
lane size: 10, top 3 accuarcy_2: 0.8395924308588064%, 3435
lane size: 11, top 3 accuarcy_1: 0.9230941704035872%, 4460
lane size: 11, top 3 accuarcy_2: 0.7838565022421524%, 4460
lane size: 12, top 3 accuarcy_1: 0.9089869474348355%, 8453
lane size: 12, top 3 accuarcy_2: 0.747426949012185%, 8453
lane size: 13, top 3 accuarcy_1: 0.9009157509157512%, 1820
lane size: 13, top 3 accuarcy_2: 0.7263736263736263%, 1820
lane size: 14, top 3 accuarcy_1: 0.8785564540281517%, 4452
lane size: 14, top 3 accuarcy_2: 0.672282120395328%, 4452


In [23]:
for i, (x, y) in enumerate(test_data_saved[0]):
    print(x.shape)
for i, (x, y) in enumerate(test_data_saved[1]):
    print(x.shape)

torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([32, 8, 9])
torch.Size([8, 8, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([32, 9, 9])
torch.Size([12, 9, 9])
